# Final Project Submission

- Student Name: Deztany Jackson
- Student Pace: Self Pace
- Scheduled Project Review: April 7, 2023
- Instructor Name: Morgan Jones



Main Library Elemenets:
[Checklist](https://docs.google.com/document/d/1swj3BthL_M1IzJzkOmzhCWc595sRC6WrFI8S-xTKCr0/edit)

## Business Understanding

# Clean up later
- Real estate company (Primary Stakeholder) wanting to partner with the local pet adoption agency
    - Adoption, Animals, Buyers/Sellers (potential adoptees), Adoptees (potenial customers), Data Scientist
    - Wants to minimize danger and make neighbor hoods attractive while caring for animals and potentially getting customers to to think about housing needs and pet adoption at same time
Assumptions:
    - The RE is not a SME in adoption and animals, so they only care if an animal wiill be adopted by closing - Binary Classification
    [https://towardsdatascience.com/the-3-most-important-basic-classification-metrics-3368dd425f74](https://towardsdatascience.com/the-3-most-important-basic-classification-metrics-3368dd425f74)
    - Want Precision to maximize TP and minimize FP of adoptees when I have results that tell me a lot of animals will be adopted.  Dont want to sell or rent and put integrity on line...
    - RE agents care about knowing the true total when the TP is low and a higher FN.. ...Recall
Questions: 
    - What is the average time until close
    - What happens with most animals when people are evicted?
    - Why are most animals given up for adoption?
    - What area are we focused on?
Metrics:
    AUC-ROC True Positive Rate against False Positive Rate
    Minimize False Positives (Precision)

## Import Libraries

In [16]:
import os
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#Import Knn

from sklearn.metrics import ConfusionMatrixDisplay,classification_report,recall_score,accuracy_score, auc
from sklearn.metrics import precision_score, f1_score, confusion_matrix,  RocCurveDisplay
from sklearn.metrics import precision_recall_curve,average_precision_score, make_scorer
from sklearn.metrics import PrecisionRecallDisplay, PredictionErrorDisplay
from sklearn import tree

# Import tensorflow stuff from Model 4

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

## Data Import

In [17]:
cur_dir = os.getcwd()
cur_dir

'/Users/deztanyjackson/Documents/0_Flatiron/Capstone_P5'

In [49]:
# Import training csv file
train_df = pd.read_csv("data/train/train.csv")
test_df = pd.read_csv("data/test/test.csv")
state_df = pd.read_csv("data/StateLabels.csv")
color_df = pd.read_csv("data/ColorLabels.csv")
breed_df = pd.read_csv("data/BreedLabels.csv")

This reference is wrong find right one...was loan predicitons
[Loan Classification,Kamal Khumar, 2021 ](https://www.kaggle.com/code/kamalkhumar/loan-status-prediction)

## Data Understanding

### Dataset Description

In [55]:
# Come back to the Categorical Values of the AdoptionSpeed

<table style="width:100%">
<thead>
<tr>
<th style="text-align:center; font-weight: bold; font-size:14px">Feature </th>
<th style="text-align:center; font-weight: bold; font-size:14px">Description</th>
<th style="text-align:center; font-weight: bold; font-size:14px">Data Type</th> 
</tr>
</thead>
<tbody>
<td><b>Type</b></td>
<td>Type of Animal<br> (1=Dog,2=Cat)</td>
<td>Nominal Number</td>    
<tr> 
<td><b>Name</b></td>
<td>Name of Pet<br> (Empty if not named)</td>
<td>String</td>
</tr>
<tr> 
<td><b>Age</b></td>
<td>Age of pet when listed<br> (Months)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Breed</b></td>
<td>Breed1 (Primary breed),Breed2 (Secondary breed, if mixed) <br> (Ref Breedlabels dict)</td>
<td>Nominal Number:String </td>
</tr>
<tr>
<td><b>Gender</b></td>
<td>Gender of pet <br> 1=Male, 2=Female, 3=Mixed, if profile represents group of pets)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Color</b></td>
<td>Colors of pets<br>Ref Colorlabels dict</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Maturity Size</b></td>
<td>Number of Rooms <br>(1=Sm, 2=Med,3=Lg,4=XLg,0=Not Spec)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Fur Length</b></td>
<td>Fur Length<br>(1=Shrt, 2=Med,3=Lg,4=Lng,0=Not Spec) </td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Vaccinated</b></td>
<td>Pet has been vaccinated<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Dewormed</b></td>
<td>Pet has been dewormed<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Sterilized</b></td>
<td>Pet has been spayed/neutered<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Health</b></td>
<td>Health Condition<br>(1=Healthy,2=Minor Inj,3=Serious Inj,0=Not Spec)</td>
<td>Nominal</td>
</tr>
<tr>
<td><b>Quantity</b></td>
<td>Number of pets represented in profile<br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Fee</b></td>
<td>Adoption fee<br> (0=Free)</td>
<td>Cardinal Number</td>
</tr>
<tr>
<td><b>state</b></td>
<td>State location in Malaysia <br>(Ref to StateLabels dict)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>RescuerID</b></td>
<td>Unique hash ID of rescuer <br></td>
<td>String</td>
</tr>
<tr>
<td><b>VideoAmt</b></td>
<td>Total uploaded videos for pet <br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>PhotoAmt</b></td>
<td>Total uploaded videos for pet <br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Description</b></td>
<td>Profile write-up <br>(Primarily Eng, also Malay or Chinese)</td>
<td>String Object</td>  
</tr>
<tr>    
<td><b>AdoptionSpreed</b></td>
<td>Categorical speed of adoption. Lower is faster.<br></td>
<td>Nominal Number</td>  
</tr>
<tr>    
<td><b>PetID</b></td>
<td>Unique hash ID of pet profile<br></td>
<td>String Object</td> 
</tbody>
</table>

### Exploring Dataset

The dataset from Kaggle had several files that will be used in the modeling effort.<br>
- Training set (14993 entries)
- Test set (3972 entries) 
- Breed labels (307 entries)
- Color labels ( 7 entries)
- State labels (15 entries)

In [ ]:
#Copy original dataset
df = train_df

EDA CHeatsheet https://datasans.medium.com/exploratory-data-analysis-cheatsheet-everything-you-might-need-36230538484d
Review the values for the labels
Review missing values
review odd inputs for values (naan,)
review columns that are not needed
look at any imbalances
correlation map
    Do any need to be removed
histogram
check for outliers with scattering

bar graph of primary correlated values
use describe...
What insights can you get from the percentages and plots
Does anything need to be standardized?  Do we standardize categorical numbers?
What plots will we use


In [57]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [63]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Type           14993 non-null  int64  
 1   Name           13736 non-null  object 
 2   Age            14993 non-null  int64  
 3   Breed1         14993 non-null  int64  
 4   Breed2         14993 non-null  int64  
 5   Gender         14993 non-null  int64  
 6   Color1         14993 non-null  int64  
 7   Color2         14993 non-null  int64  
 8   Color3         14993 non-null  int64  
 9   MaturitySize   14993 non-null  int64  
 10  FurLength      14993 non-null  int64  
 11  Vaccinated     14993 non-null  int64  
 12  Dewormed       14993 non-null  int64  
 13  Sterilized     14993 non-null  int64  
 14  Health         14993 non-null  int64  
 15  Quantity       14993 non-null  int64  
 16  Fee            14993 non-null  int64  
 17  State          14993 non-null  int64  
 18  Rescue

In [62]:
#Review
train_df.describe()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
count,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000
mean,1.457614,10.452078,265.272594,74.009738,1.776162,2.234176,3.222837,1.882012,1.862002,1.467485,1.731208,1.558727,1.914227,1.036617,1.576069,21.259988,41346.028347,0.056760,3.889215,2.516441
std,0.498217,18.155790,60.056818,123.011575,0.681592,1.745225,2.742562,2.984086,0.547959,0.599070,0.667649,0.695817,0.566172,0.199535,1.472477,78.414548,32.444153,0.346185,3.487810,1.177265
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41324.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,2.000000,2.000000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,3.000000,2.000000
75%,2.000000,12.000000,307.000000,179.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,41401.000000,0.000000,5.000000,4.000000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,20.000000,3000.000000,41415.000000,8.000000,30.000000,4.000000


In [59]:
#Checking for Duplicates
train_df.duplicated().sum()

0

In [73]:
#Checking for Null
train_df.isna().sum()

Type                0
Name             1257
Age                 0
Breed1              0
Breed2              0
Gender              0
Color1              0
Color2              0
Color3              0
MaturitySize        0
FurLength           0
Vaccinated          0
Dewormed            0
Sterilized          0
Health              0
Quantity            0
Fee                 0
State               0
RescuerID           0
VideoAmt            0
Description        12
PetID               0
PhotoAmt            0
AdoptionSpeed       0
dtype: int64

In [ ]:
#For the null values
#Impute null Names and Description with Unknown

In [ ]:
#Check different names of Animals for variation: Names, unknown, and other wierd names...replace odd names with Unknown

In [89]:
#Rescuer ID
train_df["RescuerID"].duplicated().sum()
# Drop, want to focus on the puppy characteristics

9398

In [103]:
#Look into the duplicate rescuerIDs
resc_df = train_df.groupby("RescuerID", axis=0)
resc_df.sum()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
RescuerID,,,,,,,,,,,,,,,,,,,,
0007e457eb0583479bb888d54764911f,2,2,266,0,3,4,7,0,2,1,2,2,2,1,5,0,41326,0,1.0,4
0008ef529da74e9701db7eaaa4d9115a,2,8,614,307,3,4,0,0,4,2,3,3,4,2,3,0,82652,0,7.0,5
000a5830787b64da2d61b0bbceb7676d,8,10,1064,0,9,6,9,18,5,4,8,8,8,4,6,110,165604,0,13.0,6
000acff3d6148772579b3e623988131b,1,10,307,103,1,2,0,0,2,2,1,2,2,1,1,0,41401,0,1.0,2
001b9870d4db6d26e2b52520dfbce249,2,24,266,0,1,3,4,0,1,1,3,1,2,2,1,150,41326,0,2.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc64c6fa1aff33c6aa6523a12225ec3,2,6,266,266,3,3,0,0,2,1,1,1,1,1,2,0,41401,0,6.0,3
ffd648ffe5e574b912fcb92a8e0d5520,2,3,416,448,6,2,4,14,4,4,2,2,4,2,11,300,82652,0,25.0,7
ffe9d666deaad333538c341dc3a39d9a,3,64,819,0,7,4,9,12,5,8,5,5,5,3,9,100,123978,0,13.0,6


In [ ]:
#How do I have a table with the different duplicatds counted?

In [85]:
#Check duplicate petIDs = 0
train_df["PetID"].duplicated().sum()

0

In [74]:
#Check duplicate names, if more than emply cells what is it = 5932, not with same ids though

train_df["Name"].duplicated().sum()

5932

In [88]:
#Looking into name examples
name_df = train_df.groupby("Name", axis=0)
name_df.sum()



,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
Name,,,,,,,,,,,,,,,,,,,,
!,1,1,307,307,3,1,2,3,1,1,2,2,2,1,5,0,41401,0,1.0,2
! Med Long Fur Kittens,2,6,265,265,3,2,5,6,2,2,1,1,1,1,3,135,41326,0,14.0,2
!!! URGENT!!!,2,3,265,0,2,2,4,7,2,2,2,2,2,1,3,0,41326,0,1.0,4
!!!!,1,2,307,0,2,2,3,0,1,1,2,2,2,1,1,150,41401,0,3.0,1
!!.,1,1,307,307,3,2,3,0,1,1,2,2,2,1,3,0,41401,0,3.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ð¸ð·ð Take Me PLEASE ð,2,11,265,0,2,1,2,4,2,2,1,1,1,1,1,0,41326,0,16.0,3
ðððð,2,2,266,0,1,6,7,0,2,1,2,2,2,1,1,0,41326,0,4.0,1
ððSolid Black Kittenðð,2,2,266,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,0,7.0,2


In [ ]:
#Change PhotoAmt to int64, if float it should round numbers to whole number

In [ ]:
#Remove following columns: Description(Not needed dor analysys, Name (Not needed for analysis), RescueID

In [ ]:
#What do do with categorical numners? Like StateID...normalize state number High categorical number for state

In [ ]:
#Make table with description a separate table, will use this for understanding but not model prediciton

In [ ]:
# Count how many animals are from each state
# How many multiple, duplicates and weird names

In [ ]:
# Exploring training set

In [56]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


Feature Engineering
https://www.projectpro.io/article/8-feature-engineering-techniques-for-machine-learning/423Bin the speed categories for ease of the business problem

In [2]:
#Describe the data sources and explain why the data are suitable for the project
#Present the size of the dataset and descriptive statistics for all features used in the analysis
#Justify the inclusion of features based on their properties and relevance for the project
#Identify any limitations of the data that have implications for the project


In [ ]:
#Import labels as dictionaries, use  at end for presenting and specifically understand checking names
#

In [28]:
#If profile represents multiple bets, we will separate out or delete

## Data Preparation

## Modeling

In [ ]:
### 

### Final Model

## Evaluation

### Conclusion

### Reccomendations

### Limitations